In [1]:
# Get the openai secret key:
import getpass

secret_key = getpass.getpass('Please enter your openai key:')

In [2]:
from typing import List
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, SystemMessagePromptTemplate
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field

In [3]:
model = ChatOpenAI(api_key=secret_key, model="gpt-4o-mini", max_completion_tokens=500)

In [18]:
class Joke(BaseModel):
    joke: str = Field(description="The joke")
    punchline: str = Field(description="The punchline")
    
class Jokes[BaseModel]:
    jokes: List[Joke] = Field(description="A list of jokes")

parser = PydanticOutputParser(pydantic_object=Joke)

In [25]:
template = "Answer the user query.\n{format_instructions}\n{query}"
system_message_prompt = SystemMessagePromptTemplate.from_template(template)
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt])

print(system_message_prompt)
print(chat_prompt)

messages = chat_prompt.invoke({
    "query": "Tell me a really funny joke about pingvins?",
    "format_instructions": parser.get_format_instructions()
})

print(messages)

prompt=PromptTemplate(input_variables=['format_instructions', 'query'], input_types={}, partial_variables={}, template='Answer the user query.\n{format_instructions}\n{query}') additional_kwargs={}
input_variables=['format_instructions', 'query'] input_types={} partial_variables={} messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['format_instructions', 'query'], input_types={}, partial_variables={}, template='Answer the user query.\n{format_instructions}\n{query}'), additional_kwargs={})]
messages=[SystemMessage(content='Answer the user query.\nThe output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

In [26]:
result = model.invoke(messages)

In [27]:
try:
    joke_object = parser.parse(result.content)
    print(joke_object.joke)
    print(joke_object.punchline)
except Exception as e:
    print(e)

Why don't penguins like talking to strangers at parties?
Because they find it hard to break the ice!


In [34]:
structured_llm = model.with_structured_output(Joke)
result = structured_llm.invoke("Tell me a really funny joke about pingvins?")

In [35]:
print(result.punchline)
print(type(result))

They find it hard to break the ice!
<class '__main__.Joke'>
